In [1]:
import os
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer, LabelEncoder
from sklearn import metrics
import matplotlib.pyplot as plt

In [2]:
FASTA_PATH = "Vista_Dataset/vista_sequences.fasta"
MAX_LEN = 1000
BATCH_SIZE = 32
EPOCHS = 30
RANDOM_SEED = 42
RESULTS_DIR = 'results_fasta'
os.makedirs(RESULTS_DIR, exist_ok=True)

np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [3]:
def read_fasta(fasta_path):
    """Parse FASTA → return list of (header, sequence)."""
    seqs = []
    header = None
    seq = []

    with open(fasta_path, 'r') as f:
        for line in f:
            line = line.strip()

            if line.startswith(">"):
                if header and seq:
                    seqs.append((header, "".join(seq)))
                header = line[1:]   # remove '>'
                seq = []
            else:
                seq.append(line)

        if header and seq:
            seqs.append((header, "".join(seq)))

    return seqs

In [4]:
def infer_label(header, scenario):
    """Extract labels from FASTA header."""
    h = header.lower()

    if scenario == 1:
        if "human" in h:
            return "human"
        elif "mouse" in h:
            return "mouse"
        else:
            raise ValueError(f"Cannot infer binary label from header: {header}")

    else:  # scenario 2
        if "human_enhancer" in h:
            return "human_enhancer"
        elif "mouse_enhancer" in h:
            return "mouse_enhancer"
            return "no_enhancer"
        elif "no_enhancer" in h:
            return "no_enhancer"
        else:
            raise ValueError(f"Cannot infer multi-class label from header: {header}")

def load_fasta_as_df(fasta_path, scenario):
    data = read_fasta(fasta_path)
    rows = []

    for header, seq in data:
        label = infer_label(header, scenario)
        rows.append([header, seq, label])

    df = pd.DataFrame(rows, columns=["id", "sequence", "label"])
    return df

In [5]:
def clean_seq(s):
    s = str(s).upper()
    s = ''.join([c if c in 'ACGT' else random.choice('ACGT') for c in s])
    return s

def pad_trunc(seq, maxlen=MAX_LEN):
    if len(seq) >= maxlen:
        return seq[:maxlen]
    return seq + ("A" * (maxlen - len(seq)))


In [6]:
INT_MAP = {'A':1, 'C':3, 'G':2, 'T':4}
ATOM_MAP = {'A':70, 'C':58, 'G':78, 'T':66}
EIIP_MAP = {'A':0.1260, 'C':0.1340, 'G':0.0806, 'T':0.1335}

def encode_integer(seq):
    return np.array([INT_MAP[c] for c in seq], dtype=float)

def encode_atomic(seq):
    return np.array([ATOM_MAP[c] for c in seq], dtype=float)

def encode_eiip(seq):
    return np.array([EIIP_MAP[c] for c in seq], dtype=float)

def encode_bfdna(seq):
    L = len(seq)
    counts = {b: seq.count(b) for b in "ACGT"}
    freq = {b: counts[b] / L for b in "ACGT"}
    return np.array([freq[c] for c in seq], dtype=float)

ENCODERS = {
    'integer': encode_integer,
    'atomic': encode_atomic,
    'eiip': encode_eiip,
    'bfdna': encode_bfdna
}

In [7]:
def build_Xy(df, encoding='bfdna', scenario=1):
    enc = ENCODERS[encoding]

    df['sequence'] = df['sequence'].apply(clean_seq)
    df['sequence'] = df['sequence'].apply(lambda s: pad_trunc(s, MAX_LEN))

    X = np.stack([enc(s) for s in df['sequence']])

    if scenario == 1:
        le = LabelEncoder()
        y = le.fit_transform(df['label'])
        return X[..., np.newaxis], y, le.classes_
    else:
        lb = LabelBinarizer()
        y_bin = lb.fit_transform(df['label'])
        if y_bin.ndim == 1:
            y_bin = np.vstack([1-y_bin, y_bin]).T
        return X[..., np.newaxis], y_bin, lb.classes_



In [8]:
def minmax_scale_train_test(X_train, X_val, X_test):
    L = X_train.shape[1]
    train_flat = X_train.reshape(len(X_train), L)
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_flat)

    val_scaled = scaler.transform(X_val.reshape(len(X_val), L))
    test_scaled = scaler.transform(X_test.reshape(len(X_test), L))

    return train_scaled[..., None], val_scaled[..., None], test_scaled[..., None], scaler

# Model architectures identical to paper
def build_model_scenario1(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Bidirectional(layers.LSTM(256, return_sequences=True))(inputs)
    x = layers.Activation("selu")(x)
    x = layers.Dropout(0.15)(x)

    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
    x = layers.Activation("selu")(x)
    x = layers.Dropout(0.20)(x)

    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
    x = layers.Activation("selu")(x)
    x = layers.Dropout(0.20)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation="selu")(x)
    x = layers.Dense(256, activation="selu")(x)
    x = layers.Dense(128, activation="selu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = models.Model(inputs, outputs)
    model.compile(optimizer=optimizers.RMSprop(),
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

def build_model_scenario2(input_shape, classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(inputs)
    x = layers.Activation("selu")(x)
    x = layers.Dropout(0.15)(x)

    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
    x = layers.Activation("selu")(x)
    x = layers.Dropout(0.20)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation="selu")(x)
    x = layers.Dense(128, activation="selu")(x)
    outputs = layers.Dense(classes, activation="softmax")(x)

    model = models.Model(inputs, outputs)
    model.compile(optimizer=optimizers.Adam(),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    return model


In [9]:
def train_and_eval(X, y, scenario, encoding):
    # split 75/15/15
    if scenario == 1:
        strat = y
    else:
        strat = np.argmax(y, axis=1)

    X_train, X_tmp, y_train, y_tmp = train_test_split(X, y,
                                                      test_size=0.30,
                                                      random_state=RANDOM_SEED,
                                                      stratify=strat)

    X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp,
                                                    test_size=0.50,
                                                    random_state=RANDOM_SEED,
                                                    stratify=(y_tmp if scenario==1 else np.argmax(y_tmp,axis=1)))

    X_train_s, X_val_s, X_test_s, _ = minmax_scale_train_test(X_train, X_val, X_test)

    input_shape = X_train_s.shape[1:]

    if scenario == 1:
        model = build_model_scenario1(input_shape)
    else:
        model = build_model_scenario2(input_shape, y.shape[1])

    es = callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    ckpt = callbacks.ModelCheckpoint(
        os.path.join(RESULTS_DIR, f"{encoding}_sc{scenario}.h5"),
        save_best_only=True
    )

    history = model.fit(
        X_train_s, y_train,
        validation_data=(X_val_s, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[es, ckpt],
        verbose=2
    )

    # evaluation
    preds = model.predict(X_test_s)
    if scenario == 1:
        y_pred = (preds.ravel() >= 0.5).astype(int)
        print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
    else:
        y_pred = np.argmax(preds, axis=1)
        y_true = np.argmax(y_test, axis=1)
        print("Accuracy:", metrics.accuracy_score(y_true, y_pred))


In [10]:
SCENARIO = 1             # change to 2 for multi-class
ENCODING = "bfdna"       # options: bfdna, eiip, integer, atomic

df = load_fasta_as_df(FASTA_PATH, scenario=SCENARIO)
X, y, classes = build_Xy(df, encoding=ENCODING, scenario=SCENARIO)
train_and_eval(X, y, scenario=SCENARIO, encoding=ENCODING)


Epoch 1/30


75/75 - 188s - 3s/step - accuracy: 0.5384 - loss: 1.7535 - val_accuracy: 0.5871 - val_loss: 0.7141
Epoch 2/30
75/75 - 198s - 3s/step - accuracy: 0.5291 - loss: 0.8995 - val_accuracy: 0.5871 - val_loss: 0.7235
Epoch 3/30
75/75 - 216s - 3s/step - accuracy: 0.5229 - loss: 0.8278 - val_accuracy: 0.4129 - val_loss: 12.1409
Epoch 4/30
75/75 - 206s - 3s/step - accuracy: 0.5249 - loss: 0.7680 - val_accuracy: 0.4129 - val_loss: 13.8902
Epoch 5/30
75/75 - 218s - 3s/step - accuracy: 0.5451 - loss: 0.7323 - val_accuracy: 0.4129 - val_loss: 33.7593
Epoch 6/30
75/75 - 222s - 3s/step - accuracy: 0.5484 - loss: 0.7229 - val_accuracy: 0.4129 - val_loss: 13.2253
16/16 ━━━━━━━━━━━━━━━━━━━━ 14s 844ms/step
Accuracy: 0.587890625
